In [ ]:
!pip install konlpy
!pip install xlrd
!pip install openpyxl

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 450kB 35.4MB/s 
     |████████████████████████████████| 92kB 10.3MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
#파일 업로드
from google.colab import files

#파일 읽기
#엑셀 및 txt 파일 읽기 후에 클래스 작성
uploaded = files.upload()

Saving TF_IDF.txt to TF_IDF.txt


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from tqdm.notebook import tqdm
from openpyxl import Workbook
import numpy as np
import pandas as pd
import copy


In [ ]:
#TF IDF 를 이용한  Cosine 유사도 판별

class Similarity:

  ######################
  #  코사인 유사도 판별
  #  text : 질문
  #  questionList : DaraFrame의 질문목록
  #  excelDF : 읽어온 엑셀 DataFrame
  #######################

  def __init__(self, text, excelDF):
    self.text = text # 사용자 질문
    self.excelDF = excelDF
    self.setQuestion() #question 벡터화를 위한 question 리스트

  #question 초기화 함수
  def setQuestion(self):
    question = []
    df_np = pd.DataFrame.to_numpy(self.excelDF)
    for i in range(1, len(df_np)):
      question.append(df_np[i][0])
    self.questionList = question
    
    
  def vecotrization(self): #벡터화

    ## 후에 DB에서 해당 벡터요소를 가져오는 작업 추가 

    vectorizer = TfidfVectorizer() 
    TFIDF_matrix = copy.deepcopy(self.questionList) # 리스트 깊은 복사 수행
    TFIDF_matrix.append(self.text) # 리스트에 추가
    text_matrix = vectorizer.fit_transform(TFIDF_matrix) #벡터화 실시

    return text_matrix

  def get_recommendation(self):
    new_matrix = self.vecotrization() #새로운 메트릭스 생성
    cosine_sim = linear_kernel(new_matrix, new_matrix) # 코사인 유사도 계산

    # 수정 필요 ##

    #추가된 마지막 요소의 cosine_sim 의 list 가져온다
    sim_scores = list(enumerate(cosine_sim[-1]))

    # 유사도에 따라질문들을 정렬.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 질문의 인덱스
    test_ind = [i[0] for i in sim_scores]

    print(self.excelDF['질문'].iloc[test_ind])

    # 가장 유사한 10개의 답변을 리턴
    return self.excelDF['답변'].iloc[test_ind]


  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
def main():

  path = '/content/sample_data_4600_processed.xlsx'
  df = pd.read_excel(path)

  text = "피임약 복용중 여드름이 많이 나요"

  s = Similarity(text, df)
  a = s.get_recommendation()
  print(a) #답변 출력

if __name__ == '__main__':
  main()


4155     지금 고 1이고요 초경은 중 2 때 시작했습니다 최근 들어 생리주기가 너무 들쑥날...
4372                          차이가 뭐죠? 똑같은 증상에 먹어도 되는 약인가요
2609     피도 마니 나오고 빈혈과 두통까지 온다고 하는데 왜 이런 현상이 일어나는 지 궁금...
1353     피임약을 5개월 정도 복용하였습니다. 휴약기 4~5일 정도에 생리를 하였고, 36...
1240     피임약을 깜빡했는데 원래 수요일에 먹어야 한다면 이틀 뒤에 먹으면 약 효과과 없나...
4455     색깔과 모양은 모두 같은데 제가 먹는 약은 회색 부분(SRM) 빨간 부분(HT) ...
382      하루도 빠짐없이 잘 복용했다면 계속 피임 효과가 있는 거죠? 첫 번째 팩 51-5...
990      경구피임약은 두 번째 복용 중이고 이번에 생리 전이 아니고 생리 끝난 후 2일 뒤...
4235     올해 37살입니다 주부 고요 가슴이 콕콕 쑤시면서 아프네요 가슴이랑 겨드랑이 아파...
3765     * 개인정보보호를 위하여 상담 글에 실명을 사용하지 마십시오. 주요 증상, 의 심...
Name: 질문, dtype: object
4155     프*기 노바는 에스트로겐 성분의 호르몬제로서 피임이나 배란 억제의 효과는 거의 없...
4372     콜*은 장이 예민한 사람에게는 설사를 유발할 수 있습니다. 따라서 먹는다면 진경제...
2609     1. 24시간 안에 드시면 95% 이상 효과를 기대할 수 있습니다. 2. 생리를 ...
1353     1. 다낭성 난 소나 다른 성 호르몬 이상을 일으킬 만한 산부인과적 문제가 뚜렷하...
1240     생리하기 전에 피부 트러블이 잦은 경우라면 여성 호르몬 변동에 따른 주기적 피지 ...
4455     여성들의 질 내부에는 정상 세균총이 일정하게 분포하면서 질 내부의 적정 산도 유지...
382      일단 피임약을 정확하게 잘 복용하고 계신 것을 칭찬드리고 싶습니다. 날